In [ ]:
import pathlib
import sys

In [ ]:
import pandas as pd

In [ ]:
root_dir = "../.."

In [ ]:
root_dir_path = pathlib.Path(root_dir)
data_dir_path = root_dir_path / "data"
interim_dir_path = data_dir_path / "interim"
interim_path = interim_dir_path / "interim.parquet"
processed_dir_path = data_dir_path / "processed"
train_path = processed_dir_path / "train.parquet"
src_dir_path = root_dir_path / "src"

In [ ]:
sys.path.append(str(src_dir_path))

In [ ]:
from package.constants import *
from package.feature_extraction import *
from package.utils import *

In [ ]:
interim = pd.read_parquet(interim_path)

In [ ]:
interim.dropna(inplace=True, subset=[target])

In [ ]:
create_lag_features(interim, target)

In [ ]:
import numpy as np

# Compute sample_weight
isnull = interim["sell_price"].isnull()
grouped = interim.groupby(by)
interim["sample_weight"] = grouped[target].diff()
interim.loc[isnull, "sample_weight"] = np.nan
interim["sample_weight"] **= 2
interim["sample_weight"] = grouped["sample_weight"].transform("mean")
interim["sample_weight"] = 1.0 / interim["sample_weight"]

In [ ]:
interim.dropna(inplace=True)

In [ ]:
interim.reset_index(drop=True, inplace=True)

In [ ]:
reduce_memory_usage(interim)

In [ ]:
interim.info()

In [ ]:
interim.to_parquet(train_path)